In [1]:
import cv2, numpy as np, serial, time, tkinter as tk
from PIL import Image, ImageTk
from collections import deque

try:
    esp = serial.Serial('COM3', 115200, timeout=1)
    time.sleep(2)
except:
    esp = None

cap = cv2.VideoCapture(1)
if not cap.isOpened(): exit()

root = tk.Tk()
panel = tk.Label(root)
panel.pack()

positions = deque(maxlen=5)
lower = np.array([0, 0, 0])
upper = np.array([180, 255, 50])

def update():
    ret, frame = cap.read()
    if not ret:
        root.after(10, update)
        return
    frame = cv2.flip(frame, 1)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    h, w = frame.shape[:2]
    cx = w // 2
    dz = 40
    cv2.line(frame, (cx, 0), (cx, h), (0, 0, 255), 2)

    if cnts:
        c = max(cnts, key=cv2.contourArea)
        if cv2.contourArea(c) > 800:
            x, y, cw, ch = cv2.boundingRect(c)
            obj_x = x + cw // 2
            positions.append(obj_x)
            avg_x = int(np.mean(positions))
            if esp:
                if avg_x < cx - dz: esp.write(b'L')
                elif avg_x > cx + dz: esp.write(b'R')
                else: esp.write(b'C')

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgtk = ImageTk.PhotoImage(image=Image.fromarray(img))
    panel.imgtk = imgtk
    panel.config(image=imgtk)
    root.after(30, update)

def close():
    if cap.isOpened(): cap.release()
    if esp: esp.close()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", close)
update()
root.mainloop()


In [7]:
import matplotlib.pyplot as plt


In [5]:
pip install pyserial


Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
pip install numpy matplotlib


   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.8 MB 4.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.3/12.8 MB 3.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/12.8 MB 3.7 MB/s eta 0:00:04
   ----- ---------------------------------- 1.8/12.8 MB 2.9 MB/s eta 0:00:04
   ------- -------------------------------- 2.4/12.8 MB 2.4 MB/s eta 0:00:05
   --------- ------------------------------ 2.9/12.8 MB 2.3 MB/s eta 0:00:05
   --------- ------------------------------ 3.1/12.8 MB 2.2 MB/s eta 0:00:05
   ------------ --------------------------- 3.9/12.8 MB 2.3 MB/s eta 0:00:04
   ---------------- ----------------------- 5.2/12.8 MB 2.8 MB/s eta 0:00:03
   ------------------- -------------------- 6.3/12.8 MB 3.0 MB/s eta 0:00:03
   ---------------------- ----------------- 7.3/12.8 MB 3.2 MB/s eta 0:00:02
   ------------------------- -------------- 8.1/12.8 MB 3.2 MB/s eta 0:00:02
   ---

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
